In [1]:
import numpy as np
import scipy.linalg as spl

%matplotlib inline
import matplotlib
import matplotlib.pylab as plt

from sympy import Symbol,sin,cos,nsolve
from scipy.optimize import minimize
from scipy.optimize import root

from ipywidgets import interact
import ipywidgets as widgets

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
from IPython.core.display import display, HTML

# The polling here is to ensure that plotly.js has already been loaded before
# setting display alignment in order to avoid a race condition.
display(HTML(
    '<script>'
        'var waitForPlotly = setInterval( function() {'
            'if( typeof(window.Plotly) !== "undefined" ){'
                'MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });'
                'MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);'
                'clearInterval(waitForPlotly);'
            '}}, 250 );'
    '</script>'
))
%config InlineBackend.figure_format = 'retina'

In [2]:
# A more general aproach one K point direction
class bond:
    def __init__(self, _sublattices, _pvectors, _svectors, _coeffs):
        self.sublattices = _sublattices
        self.pvectors = _pvectors
        self.svectors = _svectors
        self.vector = np.zeros(len(_pvectors[0])) 

        self.vector = np.array(_svectors[_sublattices[0]]) - np.array(_svectors[_sublattices[1]])

        for idx, c in enumerate(_coeffs):
            self.vector += c * np.array(_pvectors[idx])
        
class system:
    def __init__(self, _svectors, _bonds, _hopping):
        self.svectors = _svectors
        self.bonds = _bonds
        self.hopping = _hopping

    def hamiltonian(self,k):
        nbnd = len(self.svectors)
        H = np.zeros((nbnd, nbnd), dtype=np.complex)

        for bnd in self.bonds:
            n = bnd.sublattices[0]
            m = bnd.sublattices[1]
            
            H[n, m] -= np.complex((self.hopping * np.exp(1j*np.vdot(k, bnd.vector))))
        return H

    def gcpath(self,_labels, _points, _resolution):
        r = _resolution   # resolution of the grid
        n = len(_labels)  # number of labels
        labels = []       # labels for the plot
        locs = []         # location of the labels
        xs = []           # x values

        for i in range(n - 1):
            labels.append(_labels[i])
            locs.append(i * r)
            for x in np.linspace(0, 1, r):
                xs.append(x * (_points[i + 1] - _points[i]) + _points[i])
                
        locs.append(r * (n - 1) - 1) # last label location
        labels.append(_labels[-1])   # last label

        return locs, labels, xs

In [3]:
#Honey comb structure
a = 0.24595   # [nm] unit cell length
t = 2.5      # [eV] nearest neighbour hopping

primitive_vectors = [[0.5 * 3 , 0.5 * np.sqrt(3.)], [0.5 * 3, -0.5 * np.sqrt(3.)]]
sublattice_vectors = [[0., 0.], [1., 0.]]
bonds = [\
    bond([0, 1],  primitive_vectors, sublattice_vectors, [0, 0]),\
    bond([1, 0],  primitive_vectors, sublattice_vectors, [0, 0]),\
    bond([0, 1],  primitive_vectors, sublattice_vectors, [0, -1.]),\
    bond([0, 1],  primitive_vectors, sublattice_vectors, [-1., 0]),\
    bond([1, 0],  primitive_vectors, sublattice_vectors, [0, 1.]),\
    bond([1, 0],  primitive_vectors, sublattice_vectors, [1., 0]),\
    ]

s = system(sublattice_vectors, bonds, t)

gamma = np.array([0, 0])
K = np.array([2 * np.pi / 3., 2 * np.pi / 3. / np.sqrt(3)]) # 2π/3 and 2π/3√3  
Kprime = np.array([2 * np.pi / 3., -2 * np.pi / 3. / np.sqrt(3)]) # 2π/3 and -2π/3√3
M = np.array([np.pi / 2., np.pi / 2.])

In [4]:
def plot_graphene_2D(symm):
    if symm=='path1':
        path = [gamma, K, Kprime, gamma]
        path_string= [r"$\Gamma$", r"$K$", r"$K^\prime$", r"$\Gamma$" ]
    else:
        path = [gamma, K, M, gamma]
        path_string= [r"$\Gamma$", r"$K$", r"$M$", r"$\Gamma$" ]


    locs, labels, kpoints = s.gcpath(path_string, path,100) # get current path

    Es = []
    for k in kpoints:
        H = s.hamiltonian(k)
        evals, evecs = spl.eigh(H)
        Es.append(evals)
    
    E=np.asanyarray(Es)

    trace0 = go.Scatter(
    #     x = dos_p[:,0],
        y = E[:,0],
        mode = 'lines',
        name = '-E',
    )
    trace1 = go.Scatter(
    #     x = dos_n[:,0],
        y = E[:,1],
        mode = 'lines',
        name = '+E',
    )

    data_band = [trace0,trace1]

    bandxaxis = go.XAxis(
        title="<b>K</b>-points",
        ticks="", 
        showticklabels=True,
        ticktext=labels,
        tickvals=locs
    )
    layout_band = dict(
        title = '<b>Band_Structure - Graphene</b>',
        xaxis = bandxaxis,
        yaxis = dict(title = '<b>E</b>'),
        showlegend=False,
        )

    fig_band = dict(data=data_band, layout=layout_band)
    py.iplot(fig_band, filename='band-graphene-2d',show_link=False)

In [5]:
plot_graphene_2D('path1')